# Kapitola 19: Naivni Bayesuv klasifikator - Spam filtr pomoci pytle slov

## Jak AI rozpozna spam pomoci jednoduchych pravdepodobnosti

---

### Co se naucite:
- Pochopit model "pytle slov" (Bag of Words)
- Implementovat spam filtr pomoci scikit-learn
- Pouzit CountVectorizer pro vektorizaci textu
- Trenovat MultinomialNB klasifikator
- Analyzovat dulezitost slov pro klasifikaci

### Proc "naivni"?
Algoritmus se "naivne" domniva, ze kazde slovo v emailu je zcela nezavisly dukaz.
Nevnima gramatiku, kontext ani poradi slov. Presto je prekvapive presny!

---

## 1. Instalace a import knihoven

In [ ]:
# Instalace knihoven pro Google Colab
!pip install scikit-learn pandas numpy matplotlib seaborn -q

print("Knihovny uspesne nainstalovany!")

In [ ]:
# Import knihoven
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Nastaveni vizualizaci
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Vsechny knihovny importovany!")
print("Tema: Naivni Bayesuv klasifikator pro spam detekci")

## 2. Model "pytle slov" (Bag of Words)

Pro Naivni Bayesuv klasifikator je text jen **"pytel slov"** (Bag of Words).

Veta: _"Vyzvedni si svou vyhru zdarma!"_

Se prevede na: `{"vyzvedni": 1, "si": 1, "svou": 1, "vyhru": 1, "zdarma": 1}`

Algoritmus nezajima poradi - kazde slovo je samostatna jednotka!

In [ ]:
# Demonstrace Bag of Words
print("=" * 60)
print("DEMONSTRACE: Bag of Words (Pytel slov)")
print("=" * 60)

# Prikladove vety
vety = [
    "Vyzvedni si svou vyhru zdarma",
    "Ahoj jak se mas sejdeme zitra",
    "Klikni zde pro slevu zdarma"
]

# Vytvoreni CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(vety)

# Zobrazeni slovniku
print("\nSlovnik (slova, ktera model zna):")
print(vectorizer.get_feature_names_out())

# Zobrazeni matice vyskytu
print("\nMatice vyskytu slov (kazdy radek = jedna veta):")
df_bow = pd.DataFrame(
    X.toarray(),
    columns=vectorizer.get_feature_names_out(),
    index=[f'Veta {i+1}' for i in range(len(vety))]
)
print(df_bow)

print("\nKazdy sloupeC = jedno slovo, hodnota = kolikrat se vyskytuje")

In [ ]:
# Vizualizace Bag of Words
plt.figure(figsize=(14, 5))

sns.heatmap(df_bow, annot=True, cmap='YlOrRd', fmt='d', cbar_kws={'label': 'Pocet vyskytu'})
plt.title('Bag of Words - Matice vyskytu slov', fontsize=14)
plt.xlabel('Slova')
plt.ylabel('Vety')

plt.tight_layout()
plt.show()

print("\nVsimni si: slovo 'zdarma' se vyskytuje ve vetach 1 a 3 - to jsou typicke spam slova!")

## 3. Priprava trenovacich dat

Vytvorime dataset emailu pro trenovani spam filtru:
- **SPAM**: Nevyzadane emaily (reklamy, podvody)
- **HAM**: Legitimni emaily

In [ ]:
# Trenovaci data pro spam filtr
data = [
    # SPAM emaily
    ('Vyzvedni si svou vyhru zdarma klikni zde', 'spam'),
    ('Exkluzivni nabidka jen pro vas nevahe jte', 'spam'),
    ('Kliknete zde pro neuveritelnou slevu', 'spam'),
    ('Ziskejte pujcku bez dolozeni prijmu rychle', 'spam'),
    ('Vyhrali jste milion dolaru gratulujeme', 'spam'),
    ('Nakupujte zdarma slevy az 90 procent', 'spam'),
    ('Pilulky na hubnuti zadarmo objednejte', 'spam'),
    ('Rychla pujcka bez ruceni kliknete', 'spam'),
    ('Zdarma iPhone kliknete a vyhrajte', 'spam'),
    ('Nabidka prace z domu vydelejte miliony', 'spam'),
    ('Kliknete pro slevu kupte ted', 'spam'),
    ('Vase konto bude zruseno kliknete overit', 'spam'),
    # HAM (legitimni) emaily
    ('Ahoj jak se mas sejdeme se zitra', 'ham'),
    ('Zprava o stavu projektu prikladam report', 'ham'),
    ('Dekuji za vas email odpovim co nejdrive', 'ham'),
    ('Potvrzeni vasi objednavky cislo 12345', 'ham'),
    ('Schuzka zitra v 10 hodin v kancelari', 'ham'),
    ('Posilam ti dokumenty k projektu', 'ham'),
    ('Kdy budes mit cas na kavu', 'ham'),
    ('Pripominam deadline projektu v patek', 'ham'),
    ('Diky za pomoc moc si toho vazim', 'ham'),
    ('Posli mi prosim cislo uctu', 'ham'),
    ('Jak dopadla ta schuzka vcera', 'ham'),
    ('Pozvanka na narozeninovou oslavu', 'ham'),
]

# Prevedeni na DataFrame
df = pd.DataFrame(data, columns=['text', 'kategorie'])

print("=" * 60)
print("DATASET PRO SPAM FILTR")
print("=" * 60)
print(f"\nCelkem emailu: {len(df)}")
print(f"\nRozdeleni:")
print(df['kategorie'].value_counts())

print("\nUkazka dat:")
print(df.head(10))

In [ ]:
# Vizualizace rozdeleni dat
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Graf 1: Pocty
ax1 = axes[0]
kategorie_counts = df['kategorie'].value_counts()
colors = ['#e74c3c', '#2ecc71']
bars = ax1.bar(kategorie_counts.index, kategorie_counts.values, color=colors, edgecolor='black')
ax1.set_xlabel('Kategorie')
ax1.set_ylabel('Pocet emailu')
ax1.set_title('Rozdeleni emailu podle kategorie')
for bar, count in zip(bars, kategorie_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.2,
             str(count), ha='center', fontsize=12, fontweight='bold')

# Graf 2: Delka emailu
ax2 = axes[1]
df['delka'] = df['text'].apply(len)
df[df['kategorie']=='spam']['delka'].hist(ax=ax2, bins=10, alpha=0.7, label='Spam', color='#e74c3c')
df[df['kategorie']=='ham']['delka'].hist(ax=ax2, bins=10, alpha=0.7, label='Ham', color='#2ecc71')
ax2.set_xlabel('Delka textu (znaky)')
ax2.set_ylabel('Pocet')
ax2.set_title('Distribuce delky emailu')
ax2.legend()

plt.tight_layout()
plt.show()

## 4. Trenovani Naivniho Bayesova klasifikatoru

Kroky:
1. Rozdeleni dat na trenovaci a testovaci
2. Vektorizace textu (Bag of Words)
3. Trenovani MultinomialNB modelu
4. Evaluace vysledku

In [ ]:
# Priprava dat
X = df['text']
y = df['kategorie']

# Rozdeleni na trenovaci a testovaci data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print("=" * 60)
print("ROZDELENI DAT")
print("=" * 60)
print(f"Trenovacich vzorku: {len(X_train)}")
print(f"Testovacich vzorku: {len(X_test)}")

# Vektorizace textu
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(f"\nPocet unikatnich slov (features): {len(vectorizer.get_feature_names_out())}")
print(f"Tvar trenovaci matice: {X_train_vec.shape}")

In [ ]:
# Trenovani modelu
print("=" * 60)
print("TRENOVANI NAIVE BAYES MODELU")
print("=" * 60)

# Vytvoreni a trenovani klasifikatoru
model = MultinomialNB()
model.fit(X_train_vec, y_train)

print("\nModel uspesne natrenovan!")

# Predikce na testovacich datech
y_pred = model.predict(X_test_vec)

# Evaluace
presnost = accuracy_score(y_test, y_pred)
print(f"\nPresnost modelu: {presnost*100:.1f}%")

print("\nDetailni report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Vizualizace Confusion Matrix
plt.figure(figsize=(8, 6))

cm = confusion_matrix(y_test, y_pred, labels=['ham', 'spam'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Ham (predikce)', 'Spam (predikce)'],
            yticklabels=['Ham (skutecnost)', 'Spam (skutecnost)'])
plt.title('Confusion Matrix - Vysledky klasifikace', fontsize=14)

plt.tight_layout()
plt.show()

print("Interpretace:")
print(f"  - True Negatives (spravne HAM): {cm[0,0]}")
print(f"  - False Positives (HAM oznacen jako SPAM): {cm[0,1]}")
print(f"  - False Negatives (SPAM oznacen jako HAM): {cm[1,0]}")
print(f"  - True Positives (spravne SPAM): {cm[1,1]}")

## 5. Predikce na novych emailech

Otestujme model na emailech, ktere nikdy predtim nevidel.

In [ ]:
# Testovani na novych emailech
print("=" * 60)
print("PREDIKCE NA NOVYCH EMAILECH")
print("=" * 60)

nove_emaily = [
    "Ahoj posles mi prosim ten report",
    "Vyhra zdarma kliknete ihned na odkaz",
    "Schuzka zitra v 9 hodin",
    "Nova nabidka pujcky bez uroku kliknete",
    "Dekuji za pozvani na vecirek",
    "Slevy az 90 procent nakupujte zdarma"
]

# Vektorizace novych emailu
nove_emaily_vec = vectorizer.transform(nove_emaily)

# Predikce kategorii
predikce = model.predict(nove_emaily_vec)

# Predikce pravdepodobnosti
pravdepodobnosti = model.predict_proba(nove_emaily_vec)

print("\nVysledky predikce:")
print("-" * 70)
for email, kat, prob in zip(nove_emaily, predikce, pravdepodobnosti):
    ham_prob = prob[0] * 100
    spam_prob = prob[1] * 100
    print(f"Email: '{email[:40]}...'")
    print(f"  -> Predikce: {kat.upper()}")
    print(f"  -> P(ham)={ham_prob:.1f}%, P(spam)={spam_prob:.1f}%")
    print()

In [ ]:
# Vizualizace predikci
fig, ax = plt.subplots(figsize=(12, 6))

# Data pro graf
x_labels = [f'Email {i+1}' for i in range(len(nove_emaily))]
ham_probs = [p[0]*100 for p in pravdepodobnosti]
spam_probs = [p[1]*100 for p in pravdepodobnosti]

x = np.arange(len(nove_emaily))
width = 0.35

bars1 = ax.bar(x - width/2, ham_probs, width, label='P(HAM)', color='#2ecc71')
bars2 = ax.bar(x + width/2, spam_probs, width, label='P(SPAM)', color='#e74c3c')

ax.set_ylabel('Pravdepodobnost (%)')
ax.set_title('Pravdepodobnosti klasifikace novych emailu', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(x_labels)
ax.legend()
ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
ax.set_ylim(0, 105)

# Pridani finalni predikce
for i, (ham, spam, pred) in enumerate(zip(ham_probs, spam_probs, predikce)):
    ax.text(i, max(ham, spam) + 2, pred.upper(), ha='center', fontweight='bold',
            color='#e74c3c' if pred == 'spam' else '#2ecc71')

plt.tight_layout()
plt.show()

## 6. Analyza dulezitosti slov

Podivejme se, ktera slova model povazuje za nejdulezitejsi
pro rozpoznani spamu a legitimnich emailu.

In [ ]:
# Analyza dulezitosti slov
print("=" * 60)
print("ANALYZA DULEZITOSTI SLOV")
print("=" * 60)

# Ziskani log-pravdepodobnosti slov pro kazdu tridu
feature_names = vectorizer.get_feature_names_out()
log_probs = model.feature_log_prob_

# Index trid
ham_idx = list(model.classes_).index('ham')
spam_idx = list(model.classes_).index('spam')

# Vytvoreni DataFrame s log-pravdepodobnostmi
df_words = pd.DataFrame({
    'slovo': feature_names,
    'log_prob_ham': log_probs[ham_idx],
    'log_prob_spam': log_probs[spam_idx]
})

# Vypocet pomeru (log odds ratio)
df_words['spam_vs_ham'] = df_words['log_prob_spam'] - df_words['log_prob_ham']

# Top spam slova
top_spam = df_words.nlargest(10, 'spam_vs_ham')
print("\nTop 10 slov indikujicich SPAM:")
for _, row in top_spam.iterrows():
    print(f"  {row['slovo']}: {row['spam_vs_ham']:.2f}")

# Top ham slova
top_ham = df_words.nsmallest(10, 'spam_vs_ham')
print("\nTop 10 slov indikujicich HAM:")
for _, row in top_ham.iterrows():
    print(f"  {row['slovo']}: {row['spam_vs_ham']:.2f}")

In [ ]:
# Vizualizace nejdulezitejsich slov
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Spam slova
top_spam_plot = df_words.nlargest(15, 'spam_vs_ham')
ax1.barh(top_spam_plot['slovo'], top_spam_plot['spam_vs_ham'], color='#e74c3c')
ax1.set_xlabel('Log Odds Ratio (spam vs ham)')
ax1.set_title('Top 15 SPAM indikatory', fontsize=14, fontweight='bold')
ax1.invert_yaxis()

# Ham slova
top_ham_plot = df_words.nsmallest(15, 'spam_vs_ham')
ax2.barh(top_ham_plot['slovo'], -top_ham_plot['spam_vs_ham'], color='#2ecc71')
ax2.set_xlabel('Log Odds Ratio (ham vs spam)')
ax2.set_title('Top 15 HAM indikatory', fontsize=14, fontweight='bold')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

print("\nInterpretace:")
print("- Slova jako 'zdarma', 'kliknete', 'pujcku' silne indikuji spam")
print("- Slova jako 'ahoj', 'prosim', 'dekuji' silne indikuji legitimni email")

## 7. TF-IDF vektorizace

Alternativa k Bag of Words je **TF-IDF** (Term Frequency - Inverse Document Frequency),
ktera davá nižší váhu častým slovům a vyšší váhu vzácným slovům.

In [ ]:
# Srovnani CountVectorizer vs TfidfVectorizer
print("=" * 60)
print("SROVNANI: CountVectorizer vs TF-IDF")
print("=" * 60)

# TF-IDF vektorizace
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Trenovani modelu s TF-IDF
model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

# Predikce
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)
presnost_tfidf = accuracy_score(y_test, y_pred_tfidf)

print(f"\nPresnost s CountVectorizer: {presnost*100:.1f}%")
print(f"Presnost s TF-IDF: {presnost_tfidf*100:.1f}%")

# Cross-validace pro robustnejsi vysledky
print("\nCross-validace (5-fold):")

# CountVectorizer
X_all_count = vectorizer.fit_transform(X)
cv_scores_count = cross_val_score(MultinomialNB(), X_all_count, y, cv=5)
print(f"  CountVectorizer: {cv_scores_count.mean()*100:.1f}% (+/- {cv_scores_count.std()*100:.1f}%)")

# TF-IDF
X_all_tfidf = tfidf_vectorizer.fit_transform(X)
cv_scores_tfidf = cross_val_score(MultinomialNB(), X_all_tfidf, y, cv=5)
print(f"  TF-IDF: {cv_scores_tfidf.mean()*100:.1f}% (+/- {cv_scores_tfidf.std()*100:.1f}%)")

In [ ]:
# Vizualizace srovnani
fig, ax = plt.subplots(figsize=(10, 6))

metody = ['CountVectorizer', 'TF-IDF']
presnosti = [cv_scores_count.mean()*100, cv_scores_tfidf.mean()*100]
std_devs = [cv_scores_count.std()*100, cv_scores_tfidf.std()*100]

colors = ['#3498db', '#9b59b6']
bars = ax.bar(metody, presnosti, color=colors, edgecolor='black', yerr=std_devs, capsize=5)

ax.set_ylabel('Presnost (%)')
ax.set_title('Srovnani metod vektorizace textu', fontsize=14)
ax.set_ylim(0, 110)

for bar, p, s in zip(bars, presnosti, std_devs):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + s + 2,
            f'{p:.1f}%', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 8. Mini-projekt: Kompletni spam filtr

Vytvorime kompletni spam filtr jako tridu, kterou lze snadno pouzit.

In [ ]:
class SpamFiltr:
    """
    Kompletni spam filtr zalozeny na Naive Bayes.
    """
    
    def __init__(self, metoda='count'):
        """
        Inicializace spam filtru.
        
        Parametry:
        - metoda: 'count' pro CountVectorizer, 'tfidf' pro TfidfVectorizer
        """
        if metoda == 'tfidf':
            self.vectorizer = TfidfVectorizer()
        else:
            self.vectorizer = CountVectorizer()
        
        self.model = MultinomialNB()
        self.je_natrenovany = False
    
    def trenuj(self, texty, labely):
        """
        Natrenuje spam filtr na datech.
        
        Parametry:
        - texty: seznam textu emailu
        - labely: seznam labelu ('spam' nebo 'ham')
        """
        X = self.vectorizer.fit_transform(texty)
        self.model.fit(X, labely)
        self.je_natrenovany = True
        print(f"Spam filtr natrenovan na {len(texty)} emailech.")
        print(f"Slovnik obsahuje {len(self.vectorizer.get_feature_names_out())} slov.")
    
    def klasifikuj(self, email, podrobne=False):
        """
        Klasifikuje email jako spam nebo ham.
        
        Parametry:
        - email: text emailu
        - podrobne: pokud True, vypise podrobnosti
        
        Vraci:
        - slovnik s vysledkem klasifikace
        """
        if not self.je_natrenovany:
            raise Exception("Filtr neni natrenovany! Zavolej nejprve trenuj()")
        
        X = self.vectorizer.transform([email])
        predikce = self.model.predict(X)[0]
        pravdepodobnosti = self.model.predict_proba(X)[0]
        
        vysledek = {
            'email': email,
            'kategorie': predikce,
            'je_spam': predikce == 'spam',
            'p_ham': pravdepodobnosti[0],
            'p_spam': pravdepodobnosti[1],
            'jistota': max(pravdepodobnosti)
        }
        
        if podrobne:
            print(f"Email: '{email[:50]}...'" if len(email) > 50 else f"Email: '{email}'")
            print(f"Kategorie: {predikce.upper()}")
            print(f"P(ham): {pravdepodobnosti[0]*100:.1f}%")
            print(f"P(spam): {pravdepodobnosti[1]*100:.1f}%")
            print(f"Jistota: {max(pravdepodobnosti)*100:.1f}%")
        
        return vysledek
    
    def klasifikuj_hromadne(self, emaily):
        """
        Klasifikuje vice emailu najednou.
        """
        return [self.klasifikuj(email) for email in emaily]


print("Trida SpamFiltr pripravena!")

In [ ]:
# Pouziti spam filtru
print("=" * 60)
print("DEMONSTRACE: SpamFiltr v akci")
print("=" * 60)

# Vytvoreni a trenovani filtru
filtr = SpamFiltr(metoda='count')
filtr.trenuj(df['text'].tolist(), df['kategorie'].tolist())

print("\n" + "=" * 60)
print("TESTOVANI FILTRU")
print("=" * 60)

# Testovaci emaily
testovaci = [
    "Ahoj kdy se uvidime na kave",
    "Zdarma vyhra kliknete na odkaz",
    "Posli mi prosim cislo na kolegu",
    "Pujcka bez uroku nabidka kliknete"
]

for email in testovaci:
    print()
    vysledek = filtr.klasifikuj(email, podrobne=True)
    print("-" * 40)

## 9. Shrnutí kapitoly

### Co jsme se naučili:

1. **Bag of Words** - model, kde text je reprezentovan jako "pytel slov" bez ohledu na poradi

2. **CountVectorizer** - prevadi text na matici vyskytu slov

3. **Naivni Bayesuv klasifikator** - "naivne" predpoklada nezavislost slov, presto velmi efektivni

4. **TF-IDF** - alternativa k Bag of Words, ktera zohlednuje dulezitost slov

5. **Evaluace** - confusion matrix, precision, recall, F1-score

### Praktické využití Naive Bayes:
- **Spam filtry** - Gmail, Outlook
- **Analyza sentimentu** - pozitivni/negativni recenze
- **Kategorizace dokumentu** - sport, politika, kultura
- **Doporucovaci systemy** - Netflix, Amazon

In [ ]:
# Finalni vizualizace - jak Naive Bayes funguje
print("=" * 60)
print("JAK NAIVE BAYES ROZHODUJE")
print("=" * 60)

# Priklad rozhodovani
priklad_email = "Vyhra zdarma kliknete"
print(f"\nEmail: '{priklad_email}'")
print("\nProces rozhodovani:")
print("1. Rozdeleni na slova: ['vyhra', 'zdarma', 'kliknete']")
print("2. Pro kazde slovo se pocita P(slovo|spam) a P(slovo|ham)")
print("3. Nasobeni pravdepodobnosti (naivni predpoklad nezavislosti)")
print("4. Porovnani: P(spam|email) vs P(ham|email)")

# Demonstrace na realnem prikladu
vysledek = filtr.klasifikuj(priklad_email)
print(f"\n5. Vysledek: P(spam)={vysledek['p_spam']*100:.1f}% > P(ham)={vysledek['p_ham']*100:.1f}%")
print(f"   => Email je klasifikovan jako {vysledek['kategorie'].upper()}")

## 10. Cviceni pro procviceni

### Cviceni 1:
Pridejte do datasetu dalsi emaily (spam i ham) a sledujte, jak se zmeni presnost modelu.

### Cviceni 2:
Porovnejte MultinomialNB s jinymi variantami (BernoulliNB, GaussianNB).

### Cviceni 3:
Implementujte filtr pro analyzu sentimentu (pozitivni/negativni recenze produktu).

In [ ]:
# Prostor pro cviceni
print("Cviceni 1: Pridejte dalsi emaily")
print("-" * 40)

# Priklad - pridejte dalsi emaily do datasetu:
nove_data = [
    ('Specialni akce slevy nakupujte', 'spam'),
    ('Posilam ti pozvanku na konferenci', 'ham'),
]

# Pridani do datasetu
df_rozsireny = pd.concat([df, pd.DataFrame(nove_data, columns=['text', 'kategorie'])], ignore_index=True)
print(f"Puvodni pocet: {len(df)}")
print(f"Rozsireny pocet: {len(df_rozsireny)}")

# Pretrenujte model a porovnejte vysledky...

---

## Dalsi kroky

V pristi kapitole se podivame na **neuronove site** - zaklad modernich AI systemu
a hluboke uceni (Deep Learning).

---

*Kapitola 19 - Naivni Bayesuv klasifikator | Kurz AI pro zacatecniky*